In [1]:
import pandas as pd
import numpy as np
import talib
import datetime

In [2]:
# 讀取資料
df = pd.read_csv("TXF 2010-2020 1min.txt", sep = ",", encoding = "utf-8", engine = "c")

In [3]:
df.columns

Index(['Datetime', 'Open', 'High', 'Low', 'Close', 'TotalVolume'], dtype='object')

In [4]:
df.head()

,Datetime,Open,High,Low,Close,TotalVolume
0,2010-01-04 08:46:00,8203,8208,8184,8189,1125
1,2010-01-04 08:47:00,8188,8195,8186,8193,411
2,2010-01-04 08:48:00,8194,8201,8193,8201,422
3,2010-01-04 08:49:00,8201,8203,8198,8201,297
4,2010-01-04 08:50:00,8199,8200,8195,8197,189


In [5]:
df.tail()

,Datetime,Open,High,Low,Close,TotalVolume
1517924,2020-12-31 23:55:00,14660,14660,14659,14659,43
1517925,2020-12-31 23:56:00,14659,14660,14658,14658,18
1517926,2020-12-31 23:57:00,14659,14659,14658,14658,24
1517927,2020-12-31 23:58:00,14658,14658,14657,14657,28
1517928,2020-12-31 23:59:00,14657,14657,14654,14655,40


In [6]:
df.dtypes

Datetime       object
Open            int64
High            int64
Low             int64
Close           int64
TotalVolume     int64
dtype: object

#### Data Preprocessing

In [7]:
df["Datetime"] = pd.to_datetime(df["Datetime"])
df["Datetime"]

0         2010-01-04 08:46:00
1         2010-01-04 08:47:00
2         2010-01-04 08:48:00
3         2010-01-04 08:49:00
4         2010-01-04 08:50:00
                  ...        
1517924   2020-12-31 23:55:00
1517925   2020-12-31 23:56:00
1517926   2020-12-31 23:57:00
1517927   2020-12-31 23:58:00
1517928   2020-12-31 23:59:00
Name: Datetime, Length: 1517929, dtype: datetime64[ns]

In [8]:
# change data type
df["TotalVolume"] = df["TotalVolume"].astype(float)

In [9]:
# sort
df.sort_values(by = "Datetime", inplace = True, ascending = True)

In [10]:
df

,Datetime,Open,High,Low,Close,TotalVolume
0,2010-01-04 08:46:00,8203,8208,8184,8189,1125.0
1,2010-01-04 08:47:00,8188,8195,8186,8193,411.0
2,2010-01-04 08:48:00,8194,8201,8193,8201,422.0
3,2010-01-04 08:49:00,8201,8203,8198,8201,297.0
4,2010-01-04 08:50:00,8199,8200,8195,8197,189.0
...,...,...,...,...,...,...
1517924,2020-12-31 23:55:00,14660,14660,14659,14659,43.0
1517925,2020-12-31 23:56:00,14659,14660,14658,14658,18.0
1517926,2020-12-31 23:57:00,14659,14659,14658,14658,24.0
1517927,2020-12-31 23:58:00,14658,14658,14657,14657,28.0


In [11]:
df = df.reset_index(drop = True)

#### Resample
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html

如何從1分K -> 5/15/30/60分K? 日K?

In [12]:
df = df.set_index(df.Datetime).drop(columns = {"Datetime"})
df.head(10)

,Open,High,Low,Close,TotalVolume
Datetime,,,,,
2010-01-04 08:46:00,8203,8208,8184,8189,1125.0
2010-01-04 08:47:00,8188,8195,8186,8193,411.0
2010-01-04 08:48:00,8194,8201,8193,8201,422.0
2010-01-04 08:49:00,8201,8203,8198,8201,297.0
2010-01-04 08:50:00,8199,8200,8195,8197,189.0
2010-01-04 08:51:00,8196,8200,8195,8197,212.0
2010-01-04 08:52:00,8197,8200,8195,8196,207.0
2010-01-04 08:53:00,8196,8199,8195,8196,174.0
2010-01-04 08:54:00,8198,8198,8196,8196,117.0


In [13]:
df.index

DatetimeIndex(['2010-01-04 08:46:00', '2010-01-04 08:47:00',
               '2010-01-04 08:48:00', '2010-01-04 08:49:00',
               '2010-01-04 08:50:00', '2010-01-04 08:51:00',
               '2010-01-04 08:52:00', '2010-01-04 08:53:00',
               '2010-01-04 08:54:00', '2010-01-04 08:55:00',
               ...
               '2020-12-31 23:50:00', '2020-12-31 23:51:00',
               '2020-12-31 23:52:00', '2020-12-31 23:53:00',
               '2020-12-31 23:54:00', '2020-12-31 23:55:00',
               '2020-12-31 23:56:00', '2020-12-31 23:57:00',
               '2020-12-31 23:58:00', '2020-12-31 23:59:00'],
              dtype='datetime64[ns]', name='Datetime', length=1517929, freq=None)

改成5分K, 15分K

In [14]:
min_5 = df.resample("5T", label = "right", closed = "right").agg(dict(zip(df.columns,["first", "max", "min", "last", "sum"])))
min_5

,Open,High,Low,Close,TotalVolume
Datetime,,,,,
2010-01-04 08:50:00,8203.0,8208.0,8184.0,8197.0,2444.0
2010-01-04 08:55:00,8196.0,8200.0,8195.0,8199.0,798.0
2010-01-04 09:00:00,8198.0,8199.0,8193.0,8195.0,778.0
2010-01-04 09:05:00,8198.0,8206.0,8187.0,8190.0,2234.0
2010-01-04 09:10:00,8191.0,8196.0,8182.0,8189.0,1838.0
...,...,...,...,...,...
2020-12-31 23:40:00,14662.0,14664.0,14648.0,14655.0,661.0
2020-12-31 23:45:00,14656.0,14661.0,14655.0,14657.0,218.0
2020-12-31 23:50:00,14658.0,14661.0,14657.0,14659.0,81.0


#### ＊特別注意*
resample後會有很多na的資料，務必刪除，否則後續做運算很容易出現nan等等。

In [15]:
min_5[min_5["Close"].isna()]

,Open,High,Low,Close,TotalVolume
Datetime,,,,,
2010-01-04 13:50:00,NaN,NaN,NaN,NaN,0.0
2010-01-04 13:55:00,NaN,NaN,NaN,NaN,0.0
2010-01-04 14:00:00,NaN,NaN,NaN,NaN,0.0
2010-01-04 14:05:00,NaN,NaN,NaN,NaN,0.0
2010-01-04 14:10:00,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...
2020-12-31 14:40:00,NaN,NaN,NaN,NaN,0.0
2020-12-31 14:45:00,NaN,NaN,NaN,NaN,0.0
2020-12-31 14:50:00,NaN,NaN,NaN,NaN,0.0


In [16]:
# 把na的地方刪掉！
min_5 = min_5[min_5["Close"].isna()==False].copy()

In [17]:
min_15 = df.resample("15T", label = "right", closed = "right").agg(dict(zip(df.columns,["first", "max", "min", "last", "sum"])))
# 把na的地方刪掉！
min_15 = min_15[min_15["Close"].isna()==False].copy()

改成30分K, 60分K等等

In [18]:
min_30 = df.resample("30T", label = "right", closed = "right", offset = "9h15min").agg(dict(zip(df.columns,["first", "max", "min", "last", "sum"])))
# 把na的地方刪掉！
min_30 = min_30[min_30["Close"].isna()==False].copy()
min_30

,Open,High,Low,Close,TotalVolume
Datetime,,,,,
2010-01-04 09:15:00,8203.0,8208.0,8181.0,8199.0,10165.0
2010-01-04 09:45:00,8199.0,8211.0,8185.0,8188.0,7116.0
2010-01-04 10:15:00,8185.0,8187.0,8101.0,8131.0,20267.0
2010-01-04 10:45:00,8130.0,8181.0,8122.0,8166.0,11166.0
2010-01-04 11:15:00,8166.0,8175.0,8158.0,8170.0,4449.0
...,...,...,...,...,...
2020-12-31 22:15:00,14683.0,14685.0,14681.0,14683.0,149.0
2020-12-31 22:45:00,14682.0,14683.0,14655.0,14665.0,2033.0
2020-12-31 23:15:00,14664.0,14672.0,14656.0,14664.0,1452.0


In [19]:
min_60 = df.resample("60T", label = "right", closed = "right", offset = "9h45min").agg(dict(zip(df.columns,["first", "max", "min", "last", "sum"])))
# 把na的地方刪掉！
min_60 = min_60[min_60["Close"].isna()==False].copy()

改成日K

In [20]:
daily = df.resample("B").agg(dict(zip(df.columns,["first", "max", "min", "last", "sum"])))
# 把na的地方刪掉！
daily = daily[daily["Close"].isna()==False].copy()
daily

,Open,High,Low,Close,TotalVolume
Datetime,,,,,
2010-01-04,8203.0,8211.0,8101.0,8166.0,83773.0
2010-01-05,8212.0,8260.0,8125.0,8178.0,107583.0
2010-01-06,8197.0,8336.0,8179.0,8320.0,119935.0
2010-01-07,8335.0,8355.0,8230.0,8259.0,120490.0
2010-01-08,8287.0,8297.0,8183.0,8278.0,104770.0
...,...,...,...,...,...
2020-12-25,14306.0,14383.0,14245.0,14298.0,100614.0
2020-12-28,14329.0,14472.0,14320.0,14471.0,97477.0
2020-12-29,14470.0,14488.0,14376.0,14467.0,102919.0


#### Pandas Basics

#### Getting

In [21]:
# get某個欄位(column)的資料
df["Close"]
# df.Close

Datetime
2010-01-04 08:46:00     8189
2010-01-04 08:47:00     8193
2010-01-04 08:48:00     8201
2010-01-04 08:49:00     8201
2010-01-04 08:50:00     8197
                       ...  
2020-12-31 23:55:00    14659
2020-12-31 23:56:00    14658
2020-12-31 23:57:00    14658
2020-12-31 23:58:00    14657
2020-12-31 23:59:00    14655
Name: Close, Length: 1517929, dtype: int64

In [22]:
# get某列(row)的資料
df[0:3]

,Open,High,Low,Close,TotalVolume
Datetime,,,,,
2010-01-04 08:46:00,8203,8208,8184,8189,1125.0
2010-01-04 08:47:00,8188,8195,8186,8193,411.0
2010-01-04 08:48:00,8194,8201,8193,8201,422.0


In [23]:
df["2010/01/04":"2010/01/05"]

,Open,High,Low,Close,TotalVolume
Datetime,,,,,
2010-01-04 08:46:00,8203,8208,8184,8189,1125.0
2010-01-04 08:47:00,8188,8195,8186,8193,411.0
2010-01-04 08:48:00,8194,8201,8193,8201,422.0
2010-01-04 08:49:00,8201,8203,8198,8201,297.0
2010-01-04 08:50:00,8199,8200,8195,8197,189.0
...,...,...,...,...,...
2010-01-05 13:41:00,8180,8182,8178,8179,326.0
2010-01-05 13:42:00,8180,8180,8178,8179,142.0
2010-01-05 13:43:00,8179,8180,8176,8178,252.0


#### Selection by label

In [24]:
df.loc[df.index[0:5]]

,Open,High,Low,Close,TotalVolume
Datetime,,,,,
2010-01-04 08:46:00,8203,8208,8184,8189,1125.0
2010-01-04 08:47:00,8188,8195,8186,8193,411.0
2010-01-04 08:48:00,8194,8201,8193,8201,422.0
2010-01-04 08:49:00,8201,8203,8198,8201,297.0
2010-01-04 08:50:00,8199,8200,8195,8197,189.0


In [25]:
df.loc[:, ["Open", "Close"]]

,Open,Close
Datetime,,
2010-01-04 08:46:00,8203,8189
2010-01-04 08:47:00,8188,8193
2010-01-04 08:48:00,8194,8201
2010-01-04 08:49:00,8201,8201
2010-01-04 08:50:00,8199,8197
...,...,...
2020-12-31 23:55:00,14660,14659
2020-12-31 23:56:00,14659,14658
2020-12-31 23:57:00,14659,14658


In [26]:
df.loc[df.index[0:5], ["Open", "Close"]]

,Open,Close
Datetime,,
2010-01-04 08:46:00,8203,8189
2010-01-04 08:47:00,8188,8193
2010-01-04 08:48:00,8194,8201
2010-01-04 08:49:00,8201,8201
2010-01-04 08:50:00,8199,8197


In [27]:
df["Date"] = pd.to_datetime([i.date() for i in df.index])
df

,Open,High,Low,Close,TotalVolume,Date
Datetime,,,,,,
2010-01-04 08:46:00,8203,8208,8184,8189,1125.0,2010-01-04
2010-01-04 08:47:00,8188,8195,8186,8193,411.0,2010-01-04
2010-01-04 08:48:00,8194,8201,8193,8201,422.0,2010-01-04
2010-01-04 08:49:00,8201,8203,8198,8201,297.0,2010-01-04
2010-01-04 08:50:00,8199,8200,8195,8197,189.0,2010-01-04
...,...,...,...,...,...,...
2020-12-31 23:55:00,14660,14660,14659,14659,43.0,2020-12-31
2020-12-31 23:56:00,14659,14660,14658,14658,18.0,2020-12-31
2020-12-31 23:57:00,14659,14659,14658,14658,24.0,2020-12-31


In [28]:
df.loc[(df["Date"] >= "2015/01/01") & (df["Date"] <= "2015/07/01")]

,Open,High,Low,Close,TotalVolume,Date
Datetime,,,,,,
2015-01-05 08:46:00,9238,9238,9197,9198,3962.0,2015-01-05
2015-01-05 08:47:00,9198,9208,9193,9207,1422.0,2015-01-05
2015-01-05 08:48:00,9207,9208,9201,9201,734.0,2015-01-05
2015-01-05 08:49:00,9202,9213,9200,9206,964.0,2015-01-05
2015-01-05 08:50:00,9206,9207,9201,9202,739.0,2015-01-05
...,...,...,...,...,...,...
2015-07-01 13:41:00,9286,9288,9285,9287,498.0,2015-07-01
2015-07-01 13:42:00,9288,9290,9287,9289,529.0,2015-07-01
2015-07-01 13:43:00,9289,9292,9289,9291,540.0,2015-07-01


In [29]:
df.loc[(df.index == "2015/1/5 8:46"), "Close"]

Datetime
2015-01-05 08:46:00    9198
Name: Close, dtype: int64

Selection by position

In [30]:
df.iloc[4]

Open                          8199
High                          8200
Low                           8195
Close                         8197
TotalVolume                  189.0
Date           2010-01-04 00:00:00
Name: 2010-01-04 08:50:00, dtype: object

In [31]:
df.iloc[0:5, 0:3]

,Open,High,Low
Datetime,,,
2010-01-04 08:46:00,8203,8208,8184
2010-01-04 08:47:00,8188,8195,8186
2010-01-04 08:48:00,8194,8201,8193
2010-01-04 08:49:00,8201,8203,8198
2010-01-04 08:50:00,8199,8200,8195


In [32]:
df.iloc[[1,3,5], [2, 4]]

,Low,TotalVolume
Datetime,,
2010-01-04 08:47:00,8186,411.0
2010-01-04 08:49:00,8198,297.0
2010-01-04 08:51:00,8195,212.0


In [33]:
df.iloc[:, 2:4]

,Low,Close
Datetime,,
2010-01-04 08:46:00,8184,8189
2010-01-04 08:47:00,8186,8193
2010-01-04 08:48:00,8193,8201
2010-01-04 08:49:00,8198,8201
2010-01-04 08:50:00,8195,8197
...,...,...
2020-12-31 23:55:00,14659,14659
2020-12-31 23:56:00,14658,14658
2020-12-31 23:57:00,14658,14658


In [34]:
df.iloc[3, 4]

297.0

#### Boolean indexing

In [35]:
df[df["Close"]>10000]

,Open,High,Low,Close,TotalVolume,Date
Datetime,,,,,,
2015-04-27 09:57:00,9995,10018,9995,10018,5220.0,2015-04-27
2015-04-27 09:58:00,10018,10020,10011,10018,1636.0,2015-04-27
2015-04-27 09:59:00,10017,10026,10017,10023,1968.0,2015-04-27
2015-04-27 10:00:00,10023,10029,10020,10028,1598.0,2015-04-27
2015-04-27 10:01:00,10028,10028,10012,10012,1749.0,2015-04-27
...,...,...,...,...,...,...
2020-12-31 23:55:00,14660,14660,14659,14659,43.0,2020-12-31
2020-12-31 23:56:00,14659,14660,14658,14658,18.0,2020-12-31
2020-12-31 23:57:00,14659,14659,14658,14658,24.0,2020-12-31


In [36]:
df[(df["Close"]>10000) & (df["Open"]<10000)]

,Open,High,Low,Close,TotalVolume,Date
Datetime,,,,,,
2015-04-27 09:57:00,9995,10018,9995,10018,5220.0,2015-04-27
2015-04-27 10:44:00,9998,10006,9993,10003,1372.0,2015-04-27
2015-04-28 08:51:00,9998,10007,9998,10007,660.0,2015-04-28
2015-04-28 09:11:00,9998,10012,9994,10003,1465.0,2015-04-28
2015-04-28 09:42:00,9995,10002,9995,10001,952.0,2015-04-28
...,...,...,...,...,...,...
2020-04-07 09:22:00,9992,10020,9991,10017,3297.0,2020-04-07
2020-04-07 09:27:00,9997,10005,9997,10002,390.0,2020-04-07
2020-04-08 04:17:00,9998,10001,9997,10001,14.0,2020-04-08


In [37]:
df[df["Date"].isin(["2018/11/01", "2019/11/01"])]

,Open,High,Low,Close,TotalVolume,Date
Datetime,,,,,,
2018-11-01 00:00:00,9736,9738,9736,9738,32.0,2018-11-01
2018-11-01 00:01:00,9737,9740,9737,9737,36.0,2018-11-01
2018-11-01 00:02:00,9738,9739,9735,9735,32.0,2018-11-01
2018-11-01 00:03:00,9735,9735,9725,9725,396.0,2018-11-01
2018-11-01 00:04:00,9725,9729,9724,9727,138.0,2018-11-01
...,...,...,...,...,...,...
2019-11-01 23:55:00,11389,11390,11389,11390,13.0,2019-11-01
2019-11-01 23:56:00,11390,11390,11389,11389,8.0,2019-11-01
2019-11-01 23:57:00,11389,11389,11389,11389,12.0,2019-11-01


#### Setting

In [38]:
test = df.copy()

In [39]:
test.loc[test["Close"]>10000, "Close"] = 87

In [40]:
test[test.Close==87]

,Open,High,Low,Close,TotalVolume,Date
Datetime,,,,,,
2015-04-27 09:57:00,9995,10018,9995,87,5220.0,2015-04-27
2015-04-27 09:58:00,10018,10020,10011,87,1636.0,2015-04-27
2015-04-27 09:59:00,10017,10026,10017,87,1968.0,2015-04-27
2015-04-27 10:00:00,10023,10029,10020,87,1598.0,2015-04-27
2015-04-27 10:01:00,10028,10028,10012,87,1749.0,2015-04-27
...,...,...,...,...,...,...
2020-12-31 23:55:00,14660,14660,14659,87,43.0,2020-12-31
2020-12-31 23:56:00,14659,14660,14658,87,18.0,2020-12-31
2020-12-31 23:57:00,14659,14659,14658,87,24.0,2020-12-31


In [41]:
test.iloc[0, 3] = 1000000
test

,Open,High,Low,Close,TotalVolume,Date
Datetime,,,,,,
2010-01-04 08:46:00,8203,8208,8184,1000000,1125.0,2010-01-04
2010-01-04 08:47:00,8188,8195,8186,8193,411.0,2010-01-04
2010-01-04 08:48:00,8194,8201,8193,8201,422.0,2010-01-04
2010-01-04 08:49:00,8201,8203,8198,8201,297.0,2010-01-04
2010-01-04 08:50:00,8199,8200,8195,8197,189.0,2010-01-04
...,...,...,...,...,...,...
2020-12-31 23:55:00,14660,14660,14659,87,43.0,2020-12-31
2020-12-31 23:56:00,14659,14660,14658,87,18.0,2020-12-31
2020-12-31 23:57:00,14659,14659,14658,87,24.0,2020-12-31


#### Operation

In [42]:
df["Close"].mean()

10074.903189806639

In [43]:
df["Close"].std()

1570.6549105422212

In [44]:
df["Close"].sum()

15292987724

In [45]:
df["Open"] * 1/4 + df["Close"] * 1/4

Datetime
2010-01-04 08:46:00    4098.00
2010-01-04 08:47:00    4095.25
2010-01-04 08:48:00    4098.75
2010-01-04 08:49:00    4100.50
2010-01-04 08:50:00    4099.00
                        ...   
2020-12-31 23:55:00    7329.75
2020-12-31 23:56:00    7329.25
2020-12-31 23:57:00    7329.25
2020-12-31 23:58:00    7328.75
2020-12-31 23:59:00    7328.00
Length: 1517929, dtype: float64

In [46]:
df["High"]/df["Low"]

Datetime
2010-01-04 08:46:00    1.002933
2010-01-04 08:47:00    1.001099
2010-01-04 08:48:00    1.000976
2010-01-04 08:49:00    1.000610
2010-01-04 08:50:00    1.000610
                         ...   
2020-12-31 23:55:00    1.000068
2020-12-31 23:56:00    1.000136
2020-12-31 23:57:00    1.000068
2020-12-31 23:58:00    1.000068
2020-12-31 23:59:00    1.000205
Length: 1517929, dtype: float64

In [47]:
# Example: 建立均線
df["MA60"] = df["Close"].rolling(60).mean()
df.head(60)

,Open,High,Low,Close,TotalVolume,Date,MA60
Datetime,,,,,,,
2010-01-04 08:46:00,8203,8208,8184,8189,1125.0,2010-01-04,NaN
2010-01-04 08:47:00,8188,8195,8186,8193,411.0,2010-01-04,NaN
2010-01-04 08:48:00,8194,8201,8193,8201,422.0,2010-01-04,NaN
2010-01-04 08:49:00,8201,8203,8198,8201,297.0,2010-01-04,NaN
2010-01-04 08:50:00,8199,8200,8195,8197,189.0,2010-01-04,NaN
2010-01-04 08:51:00,8196,8200,8195,8197,212.0,2010-01-04,NaN
2010-01-04 08:52:00,8197,8200,8195,8196,207.0,2010-01-04,NaN
2010-01-04 08:53:00,8196,8199,8195,8196,174.0,2010-01-04,NaN
2010-01-04 08:54:00,8198,8198,8196,8196,117.0,2010-01-04,NaN


技術指標計算 -> talib package

http://mrjbq7.github.io/ta-lib/doc_index.html

In [48]:
df["ADX"] = talib.ADX(df["High"], df["Low"], df["Close"], timeperiod = 14)

進場訊號產生

In [49]:
# Example: Close站上MA20, 同時ADX > 30 -> 做多；跌破MA20, 同時ADX > 30 -> 做空
# 改用30分K跑
min_30["MA20"] = min_30["Close"].rolling(20).mean()
min_30["ADX"] = talib.ADX(min_30["High"], min_30["Low"], min_30["Close"], timeperiod = 14)
# 如果上面沒針對nan資料處理的話，這邊就會運算不出來！

In [50]:
# Method1 - iteration
print(datetime.datetime.now())
signal = []
for i in range(len(min_30)):
    if (min_30.iloc[i, 3] > min_30.iloc[i, 5]) & (min_30.iloc[i, 6] > 30):
        signal.append(1)
    elif (min_30.iloc[i, 3] < min_30.iloc[i, 5]) & (min_30.iloc[i, 6] > 30):
        signal.append(-1)
    else:
        signal.append(0)
print(datetime.datetime.now())

2021-08-27 19:03:17.299643
2021-08-27 19:03:31.963664


In [51]:
min_30["Signal"] = signal
min_30[min_30["Signal"]!=0]

,Open,High,Low,Close,TotalVolume,MA20,ADX,Signal
Datetime,,,,,,,,
2010-01-21 10:45:00,8182.0,8185.0,8124.0,8124.0,20270.0,8230.75,31.655935,-1
2010-01-21 11:15:00,8124.0,8143.0,8112.0,8135.0,12276.0,8225.15,33.791385,-1
2010-01-21 11:45:00,8135.0,8148.0,8131.0,8144.0,7839.0,8219.30,35.524373,-1
2010-01-21 12:15:00,8144.0,8158.0,8139.0,8142.0,6331.0,8213.50,36.630397,-1
2010-01-21 12:45:00,8142.0,8152.0,8120.0,8133.0,7419.0,8207.10,37.949883,-1
...,...,...,...,...,...,...,...,...
2020-12-31 10:15:00,14693.0,14705.0,14662.0,14673.0,9805.0,14680.70,34.710419,-1
2020-12-31 10:45:00,14673.0,14679.0,14630.0,14641.0,9746.0,14679.40,33.606435,-1
2020-12-31 11:15:00,14640.0,14646.0,14604.0,14646.0,13666.0,14678.35,33.396722,-1


In [52]:
# Method2 - numpy select
buy_signal = (min_30["Close"] > min_30["MA20"])
sellshort_signal = (min_30["Close"] < min_30["MA20"])
filter_1 = (min_30["ADX"] > 30)
condition = [buy_signal & filter_1, sellshort_signal & filter_1]
choice = [1, -1]
min_30["Signal_2"] = np.select(condition, choice, default = 0)

In [53]:
set(min_30["Signal"] == min_30["Signal_2"]) 

{True}

In [54]:
# 如何描述cross over/cross under?
buy_signal = (min_30["Close"] > min_30["MA20"]) & (min_30["Close"].shift() < min_30["MA20"].shift())
sellshort_signal = (min_30["Close"] < min_30["MA20"]) & (min_30["Close"].shift() > min_30["MA20"].shift())
filter_1 = (min_30["ADX"] > 30)
condition = [buy_signal & filter_1, sellshort_signal & filter_1]
choice = [1, -1]
min_30["Signal_adj"] = np.select(condition, choice, default = 0)

In [55]:
min_30["Close"].shift(-1)

Datetime
2010-01-04 09:15:00     8188.0
2010-01-04 09:45:00     8131.0
2010-01-04 10:15:00     8166.0
2010-01-04 10:45:00     8170.0
2010-01-04 11:15:00     8172.0
                        ...   
2020-12-31 22:15:00    14665.0
2020-12-31 22:45:00    14664.0
2020-12-31 23:15:00    14657.0
2020-12-31 23:45:00    14655.0
2021-01-01 00:15:00        NaN
Name: Close, Length: 52877, dtype: float64

#### Apply

In [56]:
df.apply(np.cumsum)

,Open,High,Low,Close,TotalVolume,Date,MA60,ADX
Datetime,,,,,,,,
2010-01-04 08:46:00,8203,8208,8184,8189,1125.0,2010-01-04 00:00:00.000000000,NaN,NaN
2010-01-04 08:47:00,16391,16403,16370,16382,1536.0,2050-01-07 00:00:00.000000000,NaN,NaN
2010-01-04 08:48:00,24585,24604,24563,24583,1958.0,2090-01-10 00:00:00.000000000,NaN,NaN
2010-01-04 08:49:00,32786,32807,32761,32784,2255.0,2130-01-14 00:00:00.000000000,NaN,NaN
2010-01-04 08:50:00,40985,41007,40956,40981,2444.0,2170-01-17 00:00:00.000000000,NaN,NaN
...,...,...,...,...,...,...,...,...
2020-12-31 23:55:00,15292930764,15295535774,15290314132,15292929096,325501064.0,2132-06-14 13:19:25.751607296,1.529225e+10,3.959356e+07
2020-12-31 23:56:00,15292945423,15295550434,15290328790,15292943754,325501082.0,2183-06-14 13:19:25.751607296,1.529227e+10,3.959359e+07
2020-12-31 23:57:00,15292960082,15295565093,15290343448,15292958412,325501106.0,2234-06-14 13:19:25.751607296,1.529228e+10,3.959361e+07


In [57]:
df.apply(lambda x: x.max() - x.min())

Open                         8091
High                         8090
Low                          8091
Close                        8093
TotalVolume               10451.0
Date           4014 days 00:00:00
MA60                  8060.583333
ADX                     85.590604
dtype: object

Grouping

In [58]:
df.groupby("Date").sum()   #看每一天這些的總和(每分鐘資料的總和)

,Open,High,Low,Close,TotalVolume,MA60,ADX
Date,,,,,,,
2010-01-04,2452784,2453539,2452008,2452769,83773.0,1.969718e+06,7648.665603
2010-01-05,2456142,2457002,2455231,2456112,107583.0,2.456205e+06,10371.591214
2010-01-06,2477584,2478508,2476716,2477680,119935.0,2.473371e+06,8457.487914
2010-01-07,2491495,2492401,2490429,2491429,120490.0,2.493485e+06,10068.064734
2010-01-08,2479996,2480840,2479087,2480002,104770.0,2.478989e+06,6892.311937
...,...,...,...,...,...,...,...
2020-12-26,2644489,2644638,2644282,2644467,2218.0,2.644772e+06,4491.939261
2020-12-28,12008157,12009492,12006941,12008287,97477.0,1.200330e+07,18889.360130
2020-12-29,15811545,15812976,15809906,15811429,102919.0,1.581150e+07,28182.826404


In [59]:
df.groupby("Date").apply(lambda x: x.max()-x.min()) #在2010/1/4最高的open減最低的open = 97

,Open,High,Low,Close,TotalVolume,Date,MA60,ADX
Date,,,,,,,,
2010-01-04,97,97,102,96,2459.0,0 days,53.050000,56.175107
2010-01-05,119,116,121,119,2140.0,0 days,63.900000,44.761022
2010-01-06,154,144,152,154,2460.0,0 days,142.016667,44.555132
2010-01-07,115,116,111,116,1956.0,0 days,76.750000,42.974423
2010-01-08,97,96,110,98,2142.0,0 days,47.916667,35.431325
...,...,...,...,...,...,...,...,...
2020-12-26,26,24,42,30,672.0,0 days,17.816667,38.353323
2020-12-28,146,143,149,146,2137.0,0 days,171.016667,49.112713
2020-12-29,102,102,104,103,2278.0,0 days,70.400000,49.507434


Merge

In [60]:
data1 = pd.DataFrame([["2330", "2020-8-7", 1000], ["2330", "2020-8-8", 2000]], columns = ["ticker", "date", "data1"])
data2 = pd.DataFrame({"ticker": ["2330", "2327"], "date": ["2020-8-7", "2020-8-8"], "data2": ["Yes", "No"]})

In [61]:
data1

,ticker,date,data1
0,2330,2020-8-7,1000
1,2330,2020-8-8,2000


In [62]:
data2

,ticker,date,data2
0,2330,2020-8-7,Yes
1,2327,2020-8-8,No


In [63]:
pd.merge(data1, data2, how = "inner",on = ["ticker", "date"])

,ticker,date,data1,data2
0,2330,2020-8-7,1000,Yes


In [64]:
pd.merge(data1, data2, how = "left",on = ["ticker", "date"])

,ticker,date,data1,data2
0,2330,2020-8-7,1000,Yes
1,2330,2020-8-8,2000,NaN


In [65]:
pd.merge(data1, data2, how = "right",on = ["ticker", "date"])

,ticker,date,data1,data2
0,2330,2020-8-7,1000.0,Yes
1,2327,2020-8-8,NaN,No


In [66]:
pd.merge(data1, data2, how = "outer",on = ["ticker", "date"])

,ticker,date,data1,data2
0,2330,2020-8-7,1000.0,Yes
1,2330,2020-8-8,2000.0,NaN
2,2327,2020-8-8,NaN,No


#### Concat

In [67]:
data3 = pd.DataFrame([["2330", "2020-8-7", 3000], ["2330", "2020-8-8", 4000]], columns = ["ticker", "date", "data1"])

In [68]:
pd.concat([data1, data3])

,ticker,date,data1
0,2330,2020-8-7,1000
1,2330,2020-8-8,2000
0,2330,2020-8-7,3000
1,2330,2020-8-8,4000


In [69]:
# merge時兩個欄位撞到怎麼辦？
pd.merge(data1, data3, on = ["ticker", "date"])

,ticker,date,data1_x,data1_y
0,2330,2020-8-7,1000,3000
1,2330,2020-8-8,2000,4000


In [70]:
pd.merge(data1, data3, on = ["ticker", "date"], suffixes = ("", "_new"))

,ticker,date,data1,data1_new
0,2330,2020-8-7,1000,3000
1,2330,2020-8-8,2000,4000


#### colume改名 & 只顯示特定欄位

In [71]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'TotalVolume', 'Date', 'MA60', 'ADX'], dtype='object')

In [72]:
# 部分更名
df.rename({"Open": "O", "Close": "C"}, axis = 1, inplace = True)
df.columns

Index(['O', 'High', 'Low', 'C', 'TotalVolume', 'Date', 'MA60', 'ADX'], dtype='object')

In [73]:
# 一次更名
df.columns = ["O", "H", "L", "C", "V", "D", "MA60", "ADX"]

In [74]:
# 呈現/選取特定欄位
df_new = df[["O", "H", "L", "C"]]
df_new

,O,H,L,C
Datetime,,,,
2010-01-04 08:46:00,8203,8208,8184,8189
2010-01-04 08:47:00,8188,8195,8186,8193
2010-01-04 08:48:00,8194,8201,8193,8201
2010-01-04 08:49:00,8201,8203,8198,8201
2010-01-04 08:50:00,8199,8200,8195,8197
...,...,...,...,...
2020-12-31 23:55:00,14660,14660,14659,14659
2020-12-31 23:56:00,14659,14660,14658,14658
2020-12-31 23:57:00,14659,14659,14658,14658


In [75]:
df_new["C"] = 1

<ipython-input-75-a79c86abfb71>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["C"] = 1


In [76]:
# 善用copy()
df_new_2 = df[["O", "H", "L", "C"]].copy()
df_new_2["C"] = 1

#### 存檔

In [77]:
daily

,Open,High,Low,Close,TotalVolume
Datetime,,,,,
2010-01-04,8203.0,8211.0,8101.0,8166.0,83773.0
2010-01-05,8212.0,8260.0,8125.0,8178.0,107583.0
2010-01-06,8197.0,8336.0,8179.0,8320.0,119935.0
2010-01-07,8335.0,8355.0,8230.0,8259.0,120490.0
2010-01-08,8287.0,8297.0,8183.0,8278.0,104770.0
...,...,...,...,...,...
2020-12-25,14306.0,14383.0,14245.0,14298.0,100614.0
2020-12-28,14329.0,14472.0,14320.0,14471.0,97477.0
2020-12-29,14470.0,14488.0,14376.0,14467.0,102919.0


In [78]:
daily.reset_index().to_csv("TXF 2010-2020 daily.txt", index = None, sep = ",", encoding = "utf-8")

In [79]:
min_30.reset_index().to_csv("TXF 2010-2020 30min.txt", index = None, sep = ",", encoding = "utf-8")